In [35]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
loader = DirectoryLoader('opinion_text_va', glob="*.txt",loader_cls=TextLoader)

docs = loader.load()


In [36]:
doc_sources = [doc.metadata['source']  for doc in docs]
doc_sources

['opinion_text_nj/9474271_Amada Sanjuan v. Sch.txt',
 'opinion_text_nj/9466928_State v. Quaker Vall.txt',
 'opinion_text_nj/9466933_Allstars Auto Group,.txt',
 'opinion_text_nj/9466938_Andrews v. Frank.txt',
 'opinion_text_nj/9466913_Winters v. North Hud.txt',
 'opinion_text_nj/9505870_New Jersey Division .txt',
 'opinion_text_nj/9499575_Viktoriya Usachenok .txt',
 'opinion_text_nj/9500337_State of New Jersey .txt',
 'opinion_text_nj/9493814_Comprehensive Neuros.txt',
 'opinion_text_nj/9487834_State v. Donnie E. H.txt',
 'opinion_text_nj/9494222_American Civil Liber.txt',
 'opinion_text_nj/9466906_In re Salvaggio.txt',
 'opinion_text_nj/9501460_James Kennedy, II v..txt',
 'opinion_text_nj/9466934_Rosas v. Board of Re.txt',
 'opinion_text_nj/9988090_In the Matter of Reg.txt',
 'opinion_text_nj/9988095_New Jersey Division .txt',
 'opinion_text_nj/9488316_State v. Kalil Coope.txt',
 'opinion_text_nj/9466926_Mun Jae Kim v. Leadi.txt',
 'opinion_text_nj/9466924_In re Madison Board .txt',
 '

In [37]:
import json
with open('opinion_data_va.json', 'r') as json_file:
    dic = json.load(json_file)

with open('opinion_data_va_ct.json', 'r') as json_file:
    dic1 = json.load(json_file)  
new_docs = []
for doc in docs:
    opinion_text = doc.metadata['source']
    opinion_id = opinion_text.split('/')[-1].split('_')[0]
    opinion_name = ".".join(opinion_text.split('/')[-1].split('_')[1].split('.')[:-1])
    print(opinion_name)
    for opinion in dic:
        if str(opinion['id']) == opinion_id:
            doc.metadata['source'] = str({"url":opinion['opinion_url'], "case_name":opinion_name})
    else:
        for opinion in dic1:
            if str(opinion['id']) == opinion_id:
                doc.metadata['source'] = str({"url":opinion['opinion_url'], "case_name":opinion_name})

docs

Amada Sanjuan v. Sch
State v. Quaker Vall
Allstars Auto Group,
Andrews v. Frank
Winters v. North Hud
New Jersey Division 
Viktoriya Usachenok 
State of New Jersey 
Comprehensive Neuros
State v. Donnie E. H
American Civil Liber
In re Salvaggio
James Kennedy, II v.
Rosas v. Board of Re
In the Matter of Reg
New Jersey Division 
State v. Kalil Coope
Mun Jae Kim v. Leadi
In re Madison Board 
Palisades at Fort Le
In re Engelhart
State v. Rivera
Angelini v. Board of
Geiger v. New Jersey
Barrett v. Board of 
Fanaro v. Borough & 
Palisades at Fort Le
Christine Savage v. 
State v. Quaker Vall
Allstars Auto Group,
Christa Robey and Ma
State v. Hester
Alejandra Padilla v.
State v. Aquilina
State v. Shlawrence 
In re Swarbrick
State v. Isaiah J. K
Fanaro v. Borough & 
Players Place II Con
State, Department of
Christopher Maia v. 
IMO Registrant R.H.:
Elizabeth Board of E
David Goyco v. Progr
In re the Issuance o
C.R. v. M. T.
Winters v. North Hud
Kean Federation of T
New Jersey Manufactu
Kean Feder

[Document(metadata={'source': "{'url': 'https://www.courtlistener.com/opinion/9474271/amada-sanjuan-v-school-district-of-west-new-york-hudson-county/', 'case_name': 'Amada Sanjuan v. Sch'}"}, page_content='                                    SYLLABUS\n\nThis syllabus is not part of the Court’s opinion. It has been prepared by the Office\nof the Clerk for the convenience of the reader. It has been neither reviewed nor\napproved by the Court and may not summarize all portions of the opinion.\n\n    Amada Sanjuan v. School District of West New York (A-45-22) (087515)\n\nArgued November 29, 2023 -- Decided February 12, 2024\n\nSOLOMON, J., writing for a unanimous Court.\n\n       In this appeal, the Court considers whether N.J.S.A. 18A:6-16 limits an\narbitrator’s discretion to penalize conduct under the Tenure Employees Hearing Law\n(TEHL), N.J.S.A. 18A:6-10 to -18.1.\n\n      Defendant, the Board of Education for the Town of West New York Public\nSchools (the Board), hired plaintiff Amad

In [38]:
import os


os.environ['OPENAI_API_KEY'] = 'sk-proj-aEREtcEjUj6xS7dx6qWlT3BlbkFJGJSTclsKd1CBzSPc9FOq'
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
documents = text_splitter.split_documents(docs)
db = Chroma.from_documents(persist_directory='./VA',documents=documents,embedding=OpenAIEmbeddings(model="text-embedding-3-large"))


In [ ]:
from langchain.chains.question_answering import load_qa_chain
import json
from langchain.chains.conversational_retrieval.prompts import (
    CONDENSE_QUESTION_PROMPT,

)
from langchain.chains import (
    LLMChain, ConversationalRetrievalChain
)
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
def chat_scale_ai(query):
    prompt_template = """
                Act as an expert lawyer and use the following pieces of context of cases to formulate an answer based on them. If you don't know the answer, just say that you don't know, don't try to make up an answer.
                {context}
                Question: {question}
                Helpful Answer:"""
    try:
        with open('chat_history.json', 'r') as chat_hist_load:
                chat_hist_dict = json.load(chat_hist_load)
                chat_hist_dict_for_llm = [(x,y) for x,y in chat_hist_dict['Question']] 
        if len(chat_hist_dict) > 2:
            chat_hist_dict = {}
            with open('chat_history.json', 'w') as chat_hist_load:
                chat_hist_dict = json.dump({})      
    except:
        chat_hist_dict = {}
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    loaded_db = Chroma(persist_directory='./chromadb', embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"))
    llm = ChatOpenAI(model_name='gpt-4o',temperature=0.1)
    question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
    doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT)
    test_retriever = loaded_db.as_retriever()
    qa = ConversationalRetrievalChain(
        combine_docs_chain=doc_chain,
        retriever=test_retriever,
        question_generator=question_generator,
        return_source_documents=True
        
    )

    if len(chat_hist_dict) > 0:
        payload = qa({"question":str(query), "chat_history":chat_hist_dict_for_llm})
    else:
        payload = qa({"question":str(query), "chat_history":[]})
        
        
    if len(chat_hist_dict) == 0:
        chat_hist_dict["Question"] = []

    chat_hist_dict['Question'].append((query, payload['answer']))

    with open('chat_history.json', 'w') as chat_hist:
        json.dump(chat_hist_dict,chat_hist)
    print(payload)
    return payload['answer']

In [ ]:
ans = chat_scale_ai("What is the standard for determining child custody in Georgia?")

In [ ]:
ans

'In Georgia, the standard for determining child custody is the "best interests of the child." This standard is codified in OCGA § 19-9-3(a)(2), which requires the court to consider the best interests of the child in making any custody determination. The court may consider a variety of factors, including but not limited to:\n\n1. The love, affection, bonding, and emotional ties existing between each parent and the child.\n2. The capacity and disposition of each parent to give the child love, affection, and guidance and to continue the education and rearing of the child.\n3. Each parent\'s knowledge and familiarity of the child and the child\'s needs.\n4. The capacity and disposition of each parent to provide the child with food, clothing, medical care, day-to-day needs, and other necessary basic care.\n5. The home environment of each parent considering the promotion of nurturance and safety of the child rather than superficial or material factors.\n6. The importance of continuity in the child\'s life and the length of time the child has lived in a stable, satisfactory environment and the desirability of maintaining continuity.\n7. The stability of the family unit of each of the parents and the presence or absence of each parent\'s support systems within the community to benefit the child.\n8. The mental and physical health of each parent, except that a disability of a parent alone shall not be determinative of custody.\n9. Each parent\'s involvement, or lack thereof, in the child\'s educational, social, and extracurricular activities.\n10. Each parent\'s employment schedule and the related flexibility or limitations, if any, of a parent to care for the child.\n11. The home, school, and community record and history of the child, as well as any health or educational special needs of the child.\n12. The willingness and ability of each parent to facilitate and encourage a close and continuing parent-child relationship between the child and the other parent, consistent with the best interest of the child.\n13. Any recommendation by a court-appointed custody evaluator or guardian ad litem.\n14. Any evidence of family violence or sexual, mental, or physical child abuse or criminal history of either parent.\n15. Any evidence of substance abuse by either parent.\n\nThe court\'s primary focus is on the child\'s welfare and ensuring that the custody arrangement serves the child\'s best interests. The court may also consider the child\'s own wishes if the child is of sufficient age and maturity to express a preference.'

In [ ]:
ans = chat_scale_ai("Explain the concept of 'best interests of the child.'")
ans

'The concept of the "best interests of the child" is a legal standard used in family law to determine the most favorable arrangement for a child\'s well-being and development. This principle is paramount in decisions regarding child custody, visitation, and support. The goal is to ensure that the child\'s physical, emotional, and psychological needs are met in the best possible way.\n\nIn the context of the cases provided:\n\n1. **Child Support and Parenting Time Deviation (OCGA § 19-6-15)**:\n   - In the first case, the trial court applied a parenting time deviation to reduce the husband\'s child support obligation. However, the court failed to explain how it calculated the deviation and did not include express findings that the deviation was in the best interests of the children. According to OCGA § 19-6-15, any deviation from the presumptive amount of child support must be justified by special circumstances and must be in the best interests of the child. The court must explicitly state that the deviation will not seriously impair the parent\'s ability to provide for the child. This ensures that the child\'s needs are prioritized over the financial convenience of the parents.\n\n2. **Custody and Parental Rights (Johnson v. Smith)**:\n   - In the second case, the court dealt with the issue of custody and the surrender of parental rights. The court emphasized the importance of strict compliance with adoption statutes to protect the child\'s best interests. The intended adoption failed because Ms. Johnson did not sign the necessary documents, and the court ruled that an invalid consent to adoption cannot be converted into a valid contractual release of parental rights. This decision underscores the principle that any action affecting a child\'s custody or parental rights must be meticulously followed to ensure the child\'s best interests are safeguarded.\n\nIn summary, the "best interests of the child" standard requires courts to consider various factors, including the child\'s safety, emotional needs, stability, and the ability of each parent to meet those needs. Courts must provide clear and explicit findings to justify their decisions, ensuring that the child\'s welfare is the primary concern in any legal determination.'

{'question': 'What is the standard for determining child custody in Georgia?', 'chat_history': [('What is the standard for determining child custody in Georgia?', 'In Georgia, the standard for determining child custody is the "best interests of the child." This standard is codified in OCGA § 19-9-3(a)(2), which requires the court to consider the best interests of the child in making custody decisions. The court evaluates various factors to determine what arrangement will best serve the child\'s physical, mental, and emotional needs. These factors can include, but are not limited to:\n\n1. The love, affection, bonding, and emotional ties existing between each parent and the child.\n2. The capacity and disposition of each parent to give the child love, affection, and guidance and to continue the education and rearing of the child.\n3. Each parent\'s knowledge and familiarity of the child and the child\'s needs.\n4. The capacity and disposition of each parent to provide the child with food, clothing, medical care, day-to-day needs, and other necessary basic care.\n5. The home environment of each parent considering the promotion of nurturance and safety of the child rather than superficial or material factors.\n6. The importance of continuity in the child\'s life and the length of time the child has lived in a stable, satisfactory environment and the desirability of maintaining continuity.\n7. The stability of the family unit of each of the parents and the presence or absence of each parent\'s support systems within the community to benefit the child.\n8. The mental and physical health of each parent.\n9. Each parent\'s involvement, or lack thereof, in the child\'s educational, social, and extracurricular activities.\n10. Each parent\'s employment schedule and the related flexibility or limitations, if any, of a parent to care for the child.\n11. The home, school, and community record and history of the child, as well as any health or educational special needs of the child.\n12. The willingness and ability of each parent to facilitate and encourage a close and continuing parent-child relationship between the child and the other parent, consistent with the best interest of the child.\n13. Any recommendation by a court-appointed custody evaluator or guardian ad litem.\n14. Any evidence of family violence or sexual, mental, or physical child abuse or criminal history of either parent.\n15. Any evidence of substance abuse by either parent.\n\nThe court has broad discretion in weighing these factors and making a custody determination. The primary focus is always on what will best serve the child\'s overall welfare and development.'), ('What is law?', 'The term "law" refers to a system of rules created and enforced through social or governmental institutions to regulate behavior. It is a system that regulates and ensures that individuals or a community adhere to the will of the state. The law is established and enforced by the controlling authority, typically the government, and it is backed by the enforcement of penalties. The law shapes politics, economics, history, and society in various ways and serves as a mediator of relations between people.'), ('What is the standard for determining child custody in Georgia?', 'In Georgia, the standard for determining child custody is the "best interests of the child." This standard is codified in OCGA § 19-9-3(a)(2), which requires the court to consider the best interests of the child in making any custody determination. The court may consider a variety of factors, including but not limited to:\n\n1. The love, affection, bonding, and emotional ties existing between each parent and the child.\n2. The capacity and disposition of each parent to give the child love, affection, and guidance and to continue the education and rearing of the child.\n3. Each parent\'s knowledge and familiarity of the child and the child\'s needs.\n4. The capacity and disposition of each parent to provide the child with food, clothing, medical care, day-to-day needs, and other necessary basic care.\n5. The home environment of each parent considering the promotion of nurturance and safety of the child rather than superficial or material factors.\n6. The importance of continuity in the child\'s life and the length of time the child has lived in a stable, satisfactory environment and the desirability of maintaining continuity.\n7. The stability of the family unit of each of the parents and the presence or absence of each parent\'s support systems within the community to benefit the child.\n8. The mental and physical health of each parent, except that a disability of a parent alone shall not be determinative of custody.\n9. Each parent\'s involvement, or lack thereof, in the child\'s educational, social, and extracurricular activities.\n10. Each parent\'s employment schedule and the related flexibility or limitations, if any, of a parent to care for the child.\n11. The home, school, and community record and history of the child, as well as any health or educational special needs of the child.\n12. The willingness and ability of each parent to facilitate and encourage a close and continuing parent-child relationship between the child and the other parent, consistent with the best interest of the child.\n13. Any recommendation by a court-appointed custody evaluator or guardian ad litem.\n14. Any evidence of family violence or sexual, mental, or physical child abuse or criminal history of either parent.\n15. Any evidence of substance abuse by either parent.\n\nThe court\'s primary focus is on the child\'s welfare and ensuring that the custody arrangement serves the child\'s best interests. The court may also consider the child\'s own wishes if the child is of sufficient age and maturity to express a preference.'), ("Explain the concept of 'best interests of the child.'", 'The concept of the "best interests of the child" is a legal standard used in family law to determine the most favorable arrangement for a child\'s well-being and development. This principle is paramount in decisions regarding child custody, visitation, and support. The goal is to ensure that the child\'s physical, emotional, and psychological needs are met in the best possible way.\n\nIn the context of the cases provided:\n\n1. **Child Support and Parenting Time Deviation (OCGA § 19-6-15)**:\n   - In the first case, the trial court applied a parenting time deviation to reduce the husband\'s child support obligation. However, the court failed to explain how it calculated the deviation and did not include express findings that the deviation was in the best interests of the children. According to OCGA § 19-6-15, any deviation from the presumptive amount of child support must be justified by special circumstances and must be in the best interests of the child. The court must explicitly state that the deviation will not seriously impair the parent\'s ability to provide for the child. This ensures that the child\'s needs are prioritized over the financial convenience of the parents.\n\n2. **Custody and Parental Rights (Johnson v. Smith)**:\n   - In the second case, the court dealt with the issue of custody and the surrender of parental rights. The court emphasized the importance of strict compliance with adoption statutes to protect the child\'s best interests. The intended adoption failed because Ms. Johnson did not sign the necessary documents, and the court ruled that an invalid consent to adoption cannot be converted into a valid contractual release of parental rights. This decision underscores the principle that any action affecting a child\'s custody or parental rights must be meticulously followed to ensure the child\'s best interests are safeguarded.\n\nIn summary, the "best interests of the child" standard requires courts to consider various factors, including the child\'s safety, emotional needs, stability, and the ability of each parent to meet those needs. Courts must provide clear and explicit findings to justify their decisions, ensuring that the child\'s welfare is the primary concern in any legal determination.'), ('What is the standard for determining child custody in Georgia?', 'In Georgia, the standard for determining child custody is the "best interests of the child." This standard is codified in OCGA § 19-9-3(a)(2), which requires the court to consider the best interests of the child in making any custody determination. The court evaluates various factors to determine what arrangement will best serve the child\'s physical, mental, and emotional needs. These factors can include, but are not limited to:\n\n1. The love, affection, bonding, and emotional ties existing between each parent and the child.\n2. The capacity and disposition of each parent to give the child love, affection, and guidance and to continue the education and rearing of the child.\n3. Each parent\'s knowledge and familiarity of the child and the child\'s needs.\n4. The capacity and disposition of each parent to provide the child with food, clothing, medical care, day-to-day needs, and other necessary basic care.\n5. The home environment of each parent considering the promotion of nurturance and safety of the child rather than superficial or material factors.\n6. The importance of continuity in the child\'s life and the length of time the child has lived in a stable, satisfactory environment and the desirability of maintaining continuity.\n7. The stability of the family unit of each of the parents and the presence or absence of each parent\'s support systems within the community to benefit the child.\n8. The mental and physical health of each parent.\n9. Each parent\'s involvement, or lack thereof, in the child\'s educational, social, and extracurricular activities.\n10. Each parent\'s employment schedule and the related flexibility or limitations, if any, of a parent to care for the child.\n11. The home, school, and community record and history of the child, as well as any health or educational special needs of the child.\n12. Any evidence of family violence or sexual, mental, or physical child abuse or criminal history of either parent.\n13. Any recommendation by a court-appointed custody evaluator or guardian ad litem.\n14. Any evidence of substance abuse by either parent.\n15. The willingness and ability of each parent to facilitate and encourage a close and continuing parent-child relationship between the child and the other parent, consistent with the best interest of the child.\n\nThe court has broad discretion in weighing these factors and making a custody determination. The overriding concern is always the best interests of the child, and the court\'s decision will be guided by what arrangement will best promote the child\'s welfare and happiness.')], 'answer': 'In Georgia, the standard for determining child custody is the "best interests of the child." This standard is codified in OCGA § 19-9-3(a)(2), which requires the court to consider various factors to determine what arrangement will best serve the child\'s needs and welfare. These factors include, but are not limited to, the emotional ties between the child and each parent, the parents\' ability to provide for the child\'s needs, the child\'s home environment, and the child\'s preference if they are of sufficient age and maturity.\n\nIn the context of the cases provided, the court must also make specific written findings of fact to support its rulings, especially when deviating from presumptive child support obligations or when granting visitation rights to grandparents. For example, in the case involving the husband\'s child support obligation, the court was required to explain how it calculated the deviation and to include express findings that the deviation was in the best interests of the children and would not seriously impair the husband\'s ability to provide for them.\n\nSimilarly, in the case involving grandparent visitation, the court must consider whether the welfare of the child would be harmed without such visitation and make specific written findings of fact to support its decision. The court must also address any constitutional challenges raised, although such challenges may be deemed moot if they no longer benefit the party seeking relief.\n\nOverall, the "best interests of the child" standard is a comprehensive and flexible guideline that allows the court to consider a wide range of factors to ensure that the child\'s welfare is prioritized in custody and visitation decisions.', 'source_documents': [Document(page_content="1. The trial court's order includes a finding that husband earned gross monthly income of $5,483.56, approximately 65 percent of the parties' combined income. After determining a basic child support obligation of $1,582 for the parties' two minor children, the court calculated husband's pro rata share of the basic child support obligation to be $986.75. As evidenced in Schedule E attached to the court's order, however, the court applied a parenting time deviation of $434.66, reducing husband's monthly child support obligation to $552.09. See OCGA § 19-6-15(i)(2)(K). Husband does not on appeal challenge the court's decision to deviate from the presumptive child support obligation. Instead, he contends the trial court erred by failing to explain how the court calculated the deviation and failing to include express findings that the deviation was in the best interests of the children and would not seriously impair his ability to provide for the children. We agree.\nBecause the trial court's order in this case was entered after July 1, 2008, the revised child support guidelines in OCGA § 19-6-15, including the 2008 amendments thereto, are applicable. See Rumley-Miawama v. Miawama, 284 Ga. 811(1), 671 S.E.2d 827 (2009). The revised guidelines permit the fact-finder to deviate\nfrom the presumptive amount of child support when special circumstances make the presumptive amount of child support excessive or inadequate due to extended parenting time as set forth in the order of visitation or when the child resides with both parents equally.", metadata={'source': 'opinion_text/1327652_Turner v. Turner.txt'}), Document(page_content='We    conclude,     however,      that    Appellant’s     constitutional\n\nchallenges are moot. “Mootness is an issue of jurisdiction and thus\n\nmust be determined before a court addresses the merits of a claim.”\n\nIn the Interest of M. F., \n305 Ga. 820\n, 820 (\n828 SE2d 350\n) (2019)\n\n\n\n     welfare of the child would be harmed without such visitation, the\n     court shall consider and may find that harm to the child is\n     reasonably likely to result when, prior to the original action or\n     intervention:\n           (A) The minor child resided with the family member for\n           six months or more;\n           (B) The family member provided financial support for the\n           basic needs of the child for at least one year;\n           (C) There was an established pattern of regular visitation\n           or child care by the family member with the child; or\n           (D) Any other circumstance exists indicating that\n           emotional or physical harm would be reasonably likely to\n           result if such visitation is not granted.\n           The court shall make specific written findings of fact in\n     support of its rulings.\n     (2)   . . . . After visitation rights have been granted to any\n     grandparent, the . . . parent of the child may petition the court for\n     revocation or amendment of such visitation rights, for good cause\n     shown, which the court, in its discretion, may grant or deny . . . .\n\n(Emphasis supplied.)\n                                      7\n (citation and punctuation omitted). “When the resolution of a case\n\nwould be tantamount to the determination of an abstract question\n\nnot arising upon existing facts or rights,” and “[w]hen the remedy\n\nsought in litigation no longer benefits the party seeking it, the case\n\nis moot and must be dismissed.” McAlister v. Clifton, \n313 Ga. 737\n,\n\n738 (1) (\n873 SE2d 178\n) (2022) (citation and punctuation omitted).\n\n     Here, Appellant’s constitutional challenges to the grandparent', metadata={'source': 'opinion_text/9481025_CRARY v. CLAUTICE.txt'}), Document(page_content='1.   In 2017, Appellant was granted sole legal and physical\n\ncustody of her minor child, and the father of Appellant’s minor child\n\nwas granted visitation rights. Appellees filed a petition for\n\ngrandparent visitation, and, on March 16, 2022, the trial court\n\nentered a final consent order (“Grandparent Visitation Order”),\n\nwhich permitted Appellees to visit with Appellant’s minor child\n\npursuant to an agreed-upon visitation schedule.\n\n      On November 16, 2022, Appellant filed a petition asking the\n\ntrial court to set aside and revoke the Grandparent Visitation Order,\n\nto issue a citation of contempt against Appellees for their failure to\n\ncomply with the Grandparent Visitation Order, to declare OCGA\n\n§ 19-7-3 facially unconstitutional and unconstitutional as applied,\n\nand to award Appellant attorney fees and expenses.\n\n      Appellant also filed a separate motion to declare the\n\n                                   2\n grandparent visitation statute unconstitutional. Appellant argued\n\nthat the grandparent visitation statute was unconstitutional\n\nbecause it “fails to provide[,] and Georgia appellate decisions do not\n\nset forth[,]” (1) who has the burden of proof when a parent seeks to\n\nrevoke a grandparent visitation order, (2) whether proof by clear and\n\nconvincing evidence that a child would be harmed absent visitation\n\nis required for a grandparent to obtain visitation or for the court to\n\ndeny a petition to revoke visitation, (3) whether “good cause” for\n\nrevoking grandparent visitation is shown if there is an “absence of a\n\nfinding of harm,” and (4) whether courts are prevented from\n\ngranting or required to revoke grandparent visitation where the\n\nchild lives with both parents. Based on her constitutional\n\narguments, Appellant asked the trial court to vacate the\n\nGrandparent Visitation Order and “declare OCGA § 19-7-3 facially\n\nunconstitutional and [unconstitutional] as applied here.”\n\n     After Appellees answered the petition, the trial court held a', metadata={'source': 'opinion_text/9481025_CRARY v. CLAUTICE.txt'}), Document(page_content="684 S.E.2d 596 (2009)\n285 Ga. 866\nTURNER\nv.\nTURNER.\nNo. S09F1313.\nSupreme Court of Georgia.\nOctober 5, 2009.\n*597 Smith, Welch & Brittain, Pandora E. Palmer, Katherine E. Fagan, McDonough, for appellant.\nEdea M. Caldwell, McDonough, for appellee.\nTHOMPSON, Justice.\nRaymond and Jessica Turner were married in 1999 and had two children. Raymond filed for divorce in January 2008. The parties reached a partial settlement agreement which provided, inter alia, that husband and wife would share joint legal and physical custody of their two minor children, the custody arrangement being structured so husband is to have physical custody of the children from Friday a.m. until Tuesday a.m., and wife is to have physical custody from Tuesday a.m. through Friday a.m., with exceptions for holidays and other special occasions. Husband also agreed to pay wife $11,000 representing her interest in the marital residence. Left unresolved and submitted to the trial court for determination were issues of child support and the division of extracurricular expenses. The parties waived a hearing, and after an in-chambers conference, the court entered a final judgment and divorce decree which incorporated the partial settlement agreement, ordered husband to pay $552.09 in monthly child support, and apportioned the expenses for the children's extracurricular activities two-thirds to husband and one-third to wife. This Court granted husband's application for discretionary review pursuant to the Family Law Pilot Project.[1]", metadata={'source': 'opinion_text/1327652_Turner v. Turner.txt'})]}
